# Import Libraries

In [1]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_mongodb import MongoDBAtlasVectorSearch
from pymongo import MongoClient
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
import os
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
load_dotenv()

True

# LLM Model

In [2]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro-latest",
    safety_settings={
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)

In [3]:
# testing
llm.invoke("what is your name")

AIMessage(content="I am a large language model, trained by Google.  I don't have a name.\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-7a68c6c0-d2cb-486c-b1c3-18f9131f9646-0', usage_metadata={'input_tokens': 5, 'output_tokens': 21, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}})

# Retriever

In [4]:
def vector_store(model_name,embedding_dim,cluster_uri,db_name,collection_name):
    embeddings = SentenceTransformerEmbeddings(model_name=model_name)

    # initialize MongoDB python client
    client = MongoClient(cluster_uri)
    ATLAS_VECTOR_SEARCH_INDEX_NAME = "langchain-test-index-vectorstores"

    MONGODB_COLLECTION = client[db_name][collection_name]

    vector_store = MongoDBAtlasVectorSearch(
        collection=MONGODB_COLLECTION,
        embedding=embeddings,
        index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
        relevance_score_fn="cosine",
    )

    # Create vector search index on the collection
    # Since we are using the default OpenAI embedding model (ada-v2) we need to specify the dimensions as 1536
    #vector_store.create_vector_search_index(dimensions=embedding_dim)

    return vector_store

In [5]:
model_name = "all-MiniLM-L6-v2"
embedding_dim = 384
cluster_uri =os.getenv("CLUSTER_URL")
db_name = "langchain"
collection_name = "vector"


In [6]:
vector_store = vector_store(model_name=model_name,embedding_dim=embedding_dim,cluster_uri=cluster_uri,db_name=db_name,collection_name=collection_name)
vector_store.similarity_search("when to eat salad",k=3)

/tmp/ipykernel_62017/1194541438.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name=model_name)


[Document(metadata={'_id': 'd3e3bfe9-edf9-4e04-a3dc-84999f759dde', 'source': 'https://www.everydayhealth.com/diet-nutrition-pictures/best-salad-greens-for-your-health.aspx', 'head': 'Healthy Recipes', 'title': 'Ranking_14_Top_Salad_Greens_From_Best_to_Worst'}, page_content='9. RomainePortion:\xa01 cup shreddedCalories:\xa05Carbs:\xa01 gFiber:\xa00.05 gProtein:\xa00.05 gWhy It’s Healthy\xa0Romaine lettuce’s dark green color, long leaves, and crunchy texture make it a very popular salad base. Two cups of romaine bring about 30 percent of your daily vitamin A, and nearly three-quarters of your vitamin K.[18]Meal Prep Inspo“Romaine lettuce has a great crunch and is delicious served in a salad or on top of a sandwich,” says Kennedy. “Romaine can even be lightly grilled for a unique, slightly charred flavor.”To boost the nutritional value of your salad, mix romaine with some spinach or kale to pack in more antioxidants, or opt for a premixed blend. “Prepackaged salad green mixtures offer a w

In [7]:
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

In [8]:
def docs2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# History Aware Chat Prompt Template

In [9]:
contextualize_q_system_prompt = """
Given a chat history and the latest user question
which might reference context in the chat history,
formulate a standalone question which can be understood
without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is.
"""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [10]:
from langchain_core.messages import HumanMessage, AIMessage
chat_history = [
    {"role": "human", "content": "i have mole on my brest"},
                {"role": "ai", "content": "can you provide more details"}
]

In [11]:
system_prompt = (
    "You are an health doctor A.I who provide medical advices for queries. "
    "communicate as a real doctor"
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use five sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [12]:
response = rag_chain.invoke({"input":"symptoms of brest cancer?","chat_history":chat_history,"context":history_aware_retriever})

In [13]:
response["answer"]

"Breast cancer symptoms can include a lump in the breast or underarm, nipple changes (discharge, inversion, tenderness), skin changes (dimpling, redness, scaliness), and breast pain. However, many breast changes are not cancerous.  It's crucial to consult a healthcare professional for any breast concerns to ensure proper evaluation and diagnosis.  Early detection is key for successful treatment.\n"

In [14]:
response["context"][0].metadata

{'_id': '124b1da0-7f60-4286-be74-16ec1ac0ccc0',
 'source': 'https://www.everydayhealth.com/breast-cancer/guide/',
 'head': '',
 'title': 'What_Is_Breast_Cancer?_Symptoms,_Causes,_Diagnosis,_Treatment,_and_Prevention'}

In [15]:
import sqlite3
from datetime import datetime
import uuid

DB_NAME = "rag_app.db"

def get_db_connection():
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row
    return conn

def create_application_logs():
    conn = get_db_connection()
    conn.execute('''CREATE TABLE IF NOT EXISTS application_logs
    (id INTEGER PRIMARY KEY AUTOINCREMENT,
    session_id TEXT,
    user_query TEXT,
    gpt_response TEXT,
    model TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')
    conn.close()

def insert_application_logs(session_id, user_query, gpt_response, model):
    conn = get_db_connection()
    conn.execute('INSERT INTO application_logs (session_id, user_query, gpt_response, model) VALUES (?, ?, ?, ?)',
                 (session_id, user_query, gpt_response, model))
    conn.commit()
    conn.close()

def get_chat_history(session_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT user_query, gpt_response FROM application_logs WHERE session_id = ? ORDER BY created_at', (session_id,))
    messages = []
    for row in cursor.fetchall():
        messages.extend([
            {"role": "human", "content": row['user_query']},
            {"role": "ai", "content": row['gpt_response']}
        ])
    conn.close()
    return messages

# Initialize the database
create_application_logs()